In [4]:
import cv2
import numpy as np
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog

# Load Haar Cascades for face and eye detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Tkinter window setup
window = Tk()
window.title("Real-Time Face, Eye, and Hair Detection")
window.configure(bg="lightblue")

# Start/Stop toggle
is_running = False
is_paused = False

report_data = []  # For storing the details of detected colors and features

# Function to classify hair color
def classify_hair_color(hair_color):
    avg_color = np.mean(hair_color)
    if avg_color < 50:
        return "Black"
    elif avg_color < 150:
        return "Brown"
    else:
        return "White"

# Function to classify eye color
def classify_eye_color(eye_color):
    avg_color = np.mean(eye_color, axis=(0, 1))  # Average across the pixels
    if avg_color[0] > avg_color[1] and avg_color[0] > avg_color[2]:
        return "Blue"
    elif avg_color[1] > avg_color[0] and avg_color[1] > avg_color[2]:
        return "Green"
    elif avg_color[2] > avg_color[0] and avg_color[2] > avg_color[1]:
        return "Brown"
    else:
        return "Black"

# Function to classify face color
def classify_face_color(face_color):
    avg_face_color = np.mean(face_color, axis=(0, 1))
    if avg_face_color[2] > 180:  # Based on red channel value
        return "Fair"
    elif avg_face_color[1] > 150:
        return "Medium"
    else:
        return "Dark"

# Function to save report
def save_report():
    report_file = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text file", "*.txt")])
    if report_file:
        with open(report_file, 'w') as f:
            for entry in report_data:
                f.write(entry + "\n")

# Function to capture the current frame
def capture_image():
    capture_file = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG file", "*.jpg")])
    if capture_file:
        cv2.imwrite(capture_file, current_frame)
        print(f"Image saved as {capture_file}")

# Function to start video capture
def start_video():
    global is_running, is_paused
    is_running = True
    is_paused = False
    capture_video()

# Function to stop video capture
def stop_video():
    global is_running
    is_running = False

# Function to pause video capture
def pause_video():
    global is_paused
    is_paused = not is_paused

# Real-time video capture and detection
def capture_video():
    global current_frame
    if not is_running:
        return

    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)

        # Iterate over detected faces
        for (x, y, w, h) in faces:
            # Dotted rectangle around the face
            for i in range(x, x+w, 5):
                cv2.circle(frame, (i, y), 1, (255, 0, 0), -1)
                cv2.circle(frame, (i, y+h), 1, (255, 0, 0), -1)
            for i in range(y, y+h, 5):
                cv2.circle(frame, (x, i), 1, (255, 0, 0), -1)
                cv2.circle(frame, (x+w, i), 1, (255, 0, 0), -1)
            
            # Crop face region for analysis
            face_roi = frame[y:y+h, x:x+w]
            face_gray_roi = gray[y:y+h, x:x+w]
            face_color = classify_face_color(face_roi)

            # Detect eyes within the face
            eyes = eye_cascade.detectMultiScale(face_gray_roi)
            eye_colors = []
            for i, (ex, ey, ew, eh) in enumerate(eyes):
                eye_color_roi = face_roi[ey:ey+eh, ex:ex+ew]
                eye_color = classify_eye_color(eye_color_roi)
                eye_colors.append(eye_color)
                cv2.rectangle(face_roi, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                if i == 1:  # Limit to 2 eyes
                    break

            # Detect hair (above the forehead)
            hair_roi = frame[y-50:y, x:x+w] if y > 50 else frame[0:y, x:x+w]
            hair_color = classify_hair_color(hair_roi)

            # Display the detected colors on the frame
            eye_color_str = f"Eyes: {', '.join(eye_colors)}" if eye_colors else "Eyes: Not detected"
            cv2.putText(frame, f"Face: {face_color}, Hair: {hair_color}, {eye_color_str}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Append the result to the report data
            report_data.append(f"Face: {face_color}, Hair: {hair_color}, {eye_color_str}")

        # Convert the frame to PIL format for Tkinter
        current_frame = frame
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        imgtk = ImageTk.PhotoImage(image=img)

        # Update the video feed label
        video_label.imgtk = imgtk
        video_label.configure(image=imgtk)

    # Keep capturing video continuously
    video_label.after(10, capture_video)

# Video capture using OpenCV
cap = cv2.VideoCapture(0)

# Create Tkinter GUI elements
video_label = Label(window)
video_label.pack()

# Buttons to start, stop, pause, capture, and save the report
button_frame = Frame(window, bg="lightblue")
button_frame.pack(pady=10)

start_button = Button(button_frame, text="Start", command=start_video, bg="green", fg="white", padx=20)
start_button.grid(row=0, column=0, padx=10)

pause_button = Button(button_frame, text="Pause", command=pause_video, bg="yellow", padx=20)
pause_button.grid(row=0, column=1, padx=10)

stop_button = Button(button_frame, text="Stop", command=stop_video, bg="red", fg="white", padx=20)
stop_button.grid(row=0, column=2, padx=10)

capture_button = Button(button_frame, text="Capture", command=capture_image, bg="blue", fg="white", padx=20)
capture_button.grid(row=0, column=3, padx=10)

report_button = Button(button_frame, text="Save Report", command=save_report, bg="purple", fg="white", padx=20)
report_button.grid(row=0, column=4, padx=10)

# Main loop for Tkinter window
window.mainloop()

# Release the video capture object when done
cap.release()
cv2.destroyAllWindows()
